In [ ]:
#didn't want to use notebook space for package installation logs so,
#writing to a log file to see in case there is an error
!apt install -y swig >> apt_logs.txt
!pip install smac[all] >> pip_logs.txt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#ignore because sometimes MLP doesn't converge and
#it starts prompting about it, it gets annoying sometimes
#especially in the last exercise.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from typing import  List, Union, Type, Callable

%matplotlib inline

In [ ]:
BIG_SEED = 123456789 #random seed for later

from sklearn.metrics import accuracy_score, make_scorer
def weighted_score(y_true, y_pred):
    weight = [0.12 if y==0 else 0.88 for y in y_true]
    return accuracy_score(y_true, y_pred, sample_weight=weight)
score_metric = make_scorer(weighted_score)
#score_metric = 'accuracy' #to choose accuracy as the metric
label_metric = 'Weighted Accuracy'

In [ ]:
df = pd.read_csv('/kaggle/input/fertility-data-set/fertility.csv', skiprows=[0] ,
                 names=['Season','Age','Childish diseases','Accident or trauma',
                        'Surgical intervention','High fevers',
                        'Frequency of alcohol consumption','Smoking habit' ,
                        'Hours spent sitting per day','Class'])
df.head()

In [ ]:
df = pd.get_dummies(df, columns=['Season','Frequency of alcohol consumption'])

encode_col =['Childish diseases','Accident or trauma',
             'Surgical intervention','High fevers', 'Class',
             'Smoking habit', 'Hours spent sitting per day']

for col in encode_col:
    le = LabelEncoder().fit(df[col])
    df[col] = le.transform(df[col])
    
df.Class = 1 - df.Class

In [ ]:
print(len(df))
df.head()

In [ ]:
df.Class.value_counts()
#class imbalance problem!!

In [ ]:
X = df.drop('Class', axis=1).values
y = df.Class

In [ ]:
X.shape

## GP with LCB and EI

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils import shuffle

from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter,\
                                        UniformIntegerHyperparameter

from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO #Random Forest
from smac.facade.smac_bo_facade import SMAC4BO   #GP
from smac.initial_design.random_configuration_design import RandomConfigurations
from smac.optimizer import acquisition

In [ ]:
def optimizer(surrogate_model :str, 
              config_space :Type[ConfigurationSpace],
              obj_function :Type[Callable],
              acquisition_type :str,
              n_iter :int = 10,
              init_points :int = 0,
              seed :int=None) -> Union[Type[SMAC4BO], Type[SMAC4HPO]]:
    """
    It is just a small wrapper to simplify and make more intuitive 
    the SMAC optimization for ML algorithms for this assignment.
    PARAMETERS
    ----------
    @surrogate_model  can be 'gp' for GaussianProcess or 
                      'rf' for RandomForest
    @config_space     an object of type ConfigSpace containing 
                      information regarding the hyperparameters
    @obj_function     the objective function that needs to be minimized 
    @acquisition_type can be 'LCB', 'EI' or 'PI' and indicated the type 
                      of acquisition function to be used, if the value 
                      is different from 'LCB', 'EI' or 'PI' it will use 
                      the default acquisition function of SMAC.
    @n_iter           number of iterations to do, defaults to 10 iterations
    @init_points      number of initial points, defaults to 0 initial points
    @seed             fix the seed for most of the randomness to have 
                      reproducible results
    
    RETURN
    ------
    either an optimized SMAC4BO object if 'gp' model_type was chosen 
    or an optimized SMAC4HPO object if 'rf' model_type was chosen
    and in any other case an exception (ValueError) is raised
    """
    #create the scenario required by SMAC optimizers
    #First we write down it's values in a dictionary and comment the results
    if seed is None:
        deterministic = "false"
    else:
        deterministic = "true"
    scen_dict = {"run_obj": "quality",    # we optimize quality
            #the only two choises are quality and runtime
            "runcount-limit": n_iter,# number iterations
            "cs": cs,      # configuration space
            "output_dir": "logs",    # create a log folder 
            #named 'logs' to save the logs of SMAC
            "deterministic": deterministic, #if false the model
            #may need to revaluate the same points more time
            #to be sure of the value at that specific point
            "abort_on_first_run_crash": False, #continue in case of a crash instead
            #of failing the model itself, this way the model doesn't crash
            "always_race_default": False
            }
    scenario = Scenario(scenario=scen_dict)
    
    #initial random points can also be created from the 
    #configuration space itself and the seed can be fixed if necessary
    #cs.seed(seed)
    #init = cs.sample_configuration(init_points)
    
    #choose the acquisition function
    #only the one explained at the lessons are present here
    if acquisition_type == 'LCB':
        print('Using LCB acquisition Function')
        _acquistion = acquisition.LCB
    elif acquisition_type == 'EI':
        print('Using EI acquisition Function')
        _acquistion = acquisition.EI
    elif acquisition_type == 'PI':
        print('Using PI acquisition Function')
        _acquistion = acquisition.PI
    else:
        #use the defualt one chosen by SMAC based on the 
        #type of surrogate model chosen
        print('Using SMAC Default acquisition Function')
        _acquistion = None
        
    #choose and optimize the SMAC model
    if surrogate_model == 'gp':
        #use SMAC4BO
        smac = SMAC4BO(scenario=scenario, tae_runner=obj_function, 
                       rng=np.random.RandomState(seed),
                       initial_design=RandomConfigurations, 
                       initial_design_kwargs={'n_configs_x_params':init_points,
                                              'max_config_fracs':init_points/n_iter},
                       acquisition_function=_acquistion)
        _best = smac.optimize()
        print('Best Configuration found:')
        print(_best)
        return smac
    if surrogate_model == 'rf':
        #use SMAC4HPO (Bayesian Optimization with Random Forest)
        smac = SMAC4HPO(scenario=scenario, tae_runner=obj_function, 
                        rng=np.random.RandomState(seed),
                        initial_design=RandomConfigurations, 
                        initial_design_kwargs={'n_configs_x_params':init_points,
                                               'max_config_fracs':init_points/n_iter}, 
                        acquisition_function=_acquistion)
        _best = smac.optimize()
        print('Best Configuration found:')
        print(_best)
        return smac
    #in case model_type was neither gp nor rf
    raise ValueError("Model type can only 'gp' or 'rf'")

In [ ]:
#SMAC needs a Configuration Space where it will search for the hyperparameters
cs = ConfigurationSpace()

#define the domain of each hyperparameter
lr = UniformFloatHyperparameter("learning_rate_init", 0.01, 0.1, default_value=0.05)
momentum = UniformFloatHyperparameter("momentum", 0.1, 0.9, default_value=0.5)

#add the hyperparameters to the configuration space
cs.add_hyperparameters([lr, momentum])

In [ ]:
def MLP_score_1(conf: Union[Type[ConfigurationSpace], dict]) -> float:
    """
    Basically takes a configuration does a 10-CV and return the average
    score, the average score is modified so it gets minimized, for example
    accuracy is converted into error.
    PARAMETERS
    ----------
    @conf   is a ConfigurationSpace Object, but it acts as a dictionary
            so we can call a value given the key: conf['key']
            
    Returns
    -------
    @return a float value which needs to be minimized in this case cv error
    
    This function takes in the configuration and computes the error score
    using CV with 10 folds
    """
    
    MLPclf = MLPClassifier(hidden_layer_sizes=(4,2,), momentum = conf['momentum'],
                          learning_rate_init = conf['learning_rate_init'],
                          random_state=BIG_SEED)
    
    skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
    X = df.drop('Class', axis=1).values
    y = df.Class
    score = cross_val_score(MLPclf, X, y, cv=skf, scoring=score_metric)
    print("Trying lr:", conf['learning_rate_init'], "momentum:", conf['momentum'], ", got score:", np.mean(score))
    return 1 - np.mean(score) #Error metric

The ‘GP’ surrogate model (SMAC4BO) is chosen for this part, and the acquisition functions ‘EI’, ‘LCB’ and ‘PI’ are used with 5 initial points, and
the iterations are set to 25(20 + 5 initial).

The MultiLayer perceptron classifier has two hidden layers with 4 and 2 neurons respectively and all the other configuration are the default one, except
for the seed which is fixed to have consistent results

In [ ]:
#Use gp and LCB
smac_LCB = optimizer('gp', cs, obj_function=MLP_score_1, 
                     acquisition_type='LCB', n_iter=25, 
                     init_points=5, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_LCB.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Use gp and EI
smac_EI = optimizer('gp', cs, obj_function=MLP_score_1, 
                    acquisition_type='EI', n_iter=25, 
                    init_points=5, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_EI.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Use gp and PI
smac_PI = optimizer('gp', cs, obj_function=MLP_score_1, 
                    acquisition_type='PI', n_iter=25, 
                    init_points=5, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_PI.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Compare the two models removing the initial 5 points keeping only the best of them.
plt.figure(figsize=(15,5))
plt.plot(1-np.minimum.accumulate(smac_LCB.get_X_y()[1])[4:], 'o-') #0,1,2,3,4 sono i punti iniziali
plt.plot(1-np.minimum.accumulate(smac_EI.get_X_y()[1])[4:], 'o-')
plt.plot(1-np.minimum.accumulate(smac_PI.get_X_y()[1])[4:], 'o-')
plt.legend(['LCB','EI','PI'])
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.show()

## Grid and Randomized Search

For the GridSearch evenly distributed points are taken from the Configuration Domain (5 for each parameter), and RadomSearch searches random
points by itself.

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time

In [ ]:
#create the configuration grid with 5 element for each hyperparameter
conf_grid = {'learning_rate_init':np.linspace(0.01, 0.1, num=5),
             'momentum':np.linspace(0.1, 0.9, num=5)}

MLP_GS = GridSearchCV(MLPClassifier(hidden_layer_sizes=(4,2,),random_state=BIG_SEED),
                      conf_grid, scoring=score_metric, cv=10, n_jobs=-1) #n_jobs=-1 parallelize the computation

t_0 = time()
skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
X = df.drop('Class', axis=1).values
y = df.Class
MLP_GS = MLP_GS.fit(X, y)
print(MLP_GS.best_params_, "->", MLP_GS.best_score_)
print("Completed in %0.3fs" % (time() - t_0))

In [ ]:
print("Best Score", np.max(MLP_GS.cv_results_['mean_test_score']))
plt.plot(MLP_GS.cv_results_['mean_test_score'])
plt.show()

In [ ]:
param_grid = {'learning_rate_init':np.linspace(0.01, 0.1),
              'momentum':np.linspace(0.1, 0.9)}

MLP_RS = RandomizedSearchCV(MLPClassifier(hidden_layer_sizes=(4,2,), 
                                          random_state=BIG_SEED),
                            param_grid, n_iter=25, scoring=score_metric, 
                            cv=10, random_state=BIG_SEED, n_jobs=-1)

t_0 = time()
skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
X = df.drop('Class', axis=1).values
y = df.Class
MLP_RS = MLP_RS.fit(X, y)
print(MLP_RS.best_params_, "->", MLP_RS.best_score_)
print("Completed in %0.3fs" % (time() - t_0))

In [ ]:
print("Best Score", np.max(MLP_RS.cv_results_['mean_test_score']))
plt.plot(MLP_RS.cv_results_['mean_test_score'])
plt.show()

In [ ]:
plt.plot(np.maximum.accumulate(MLP_RS.cv_results_['mean_test_score']), 'o-')
plt.plot(np.maximum.accumulate(MLP_GS.cv_results_['mean_test_score']), 'o-')
plt.legend(['Random','Grid'])
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
iter_num = list(range(1,26))
lcb, = plt.plot(iter_num, 1-np.minimum.accumulate(smac_LCB.get_X_y()[1]), 'o-')
ei,  = plt.plot(iter_num, 1-np.minimum.accumulate(smac_EI.get_X_y()[1]), 'o-')
pi,  = plt.plot(iter_num, 1-np.minimum.accumulate(smac_PI.get_X_y()[1]), 'o-')
rs,  = plt.plot(iter_num, np.maximum.accumulate(MLP_RS.cv_results_['mean_test_score']), 'o-')
gs,  = plt.plot(iter_num, np.maximum.accumulate(MLP_GS.cv_results_['mean_test_score']), 'o-')
plt.legend([lcb,rs,ei, pi, gs],['LCB','Random','EI','PI','Grid']) #need to change order based on the score
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.xlim(xmin=0.5, xmax=25.5)
plt.xticks(iter_num)
plt.show()

Let's see the best model found by LCB
The grid search performs ok because this is a small dimensional hyperparameter space (2 dimensions), as we increase the dimension the more difficult it gets.

In [ ]:
best_1 = smac_LCB.get_X_y()[0][np.argmin(smac_LCB.get_X_y()[1])]

In [ ]:
from sklearn.model_selection import cross_validate

MLPclf = MLPClassifier(hidden_layer_sizes=(4, 2),
                      learning_rate_init=best_1[0], momentum=best_1[1],
                      random_state=BIG_SEED)

skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
scores = cross_validate(MLPclf, df.drop('Class', axis=1).values, df.Class, cv=skf,
                        scoring=['f1','precision','recall','f1_macro','accuracy'])

In [ ]:
print("F1:\t\t %0.3f" % (np.mean(scores['test_f1'])))
print("Precision:\t %0.3f" % (np.mean(scores['test_precision'])))
print("Recall:\t\t %0.3f" % (np.mean(scores['test_recall'])))
print("F1 Macro:\t %0.3f" % (np.mean(scores['test_f1_macro'])))
print("Accuracy:\t %0.3f" % (np.mean(scores['test_accuracy'])))

In [ ]:
best_1

First of all, it is important to notice that maximing weighted accuracy the model achieved good performance also on accuracy and that the model doesn't follow the zero rule, now we will plot the results in a contour plot to see how he searched the space, the in the next section we will use more iterations to see if we can improve the model even more.

In [ ]:
from scipy.interpolate import griddata
plt.figure(figsize=(15,10))
best_1 = smac_LCB.get_X_y()[0][np.argmin(smac_LCB.get_X_y()[1])]
x = list(map(lambda x: x[0],smac_LCB.get_X_y()[0]))
y = list(map(lambda x: x[1],smac_LCB.get_X_y()[0]))
z = 1 - smac_LCB.get_X_y()[1]
# define grid.
xi = np.linspace(0.01,0.1,50)
yi = np.linspace(0.1,0.9,50)
# grid the data.
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')
# contour the gridded data, plotting dots at the randomly spaced data points.
CS1 = plt.contour(xi,yi,zi,15,linewidths=0.5,colors='k')
CS = plt.contourf(xi,yi,zi,15,cmap=plt.cm.Blues)
#plt.colorbar()
plt.clabel(CS1, inline=1, fontsize=12)

plt.scatter(x, y, color='black')
plt.scatter(best_1[0], best_1[1], color='red', marker='s', linewidths=6)
plt.xlabel('learning rate')
plt.ylabel('momentum')
plt.title('Counter Plot')
#plt.savefig('./imgs/counter_weighted.png')
plt.show()

## Random Forest with LBC and EI

In [ ]:
hidden1 = UniformIntegerHyperparameter('h1',1,5)
hidden2 = UniformIntegerHyperparameter('h2',1,5)

cs.add_hyperparameters([hidden1, hidden2])

In [ ]:
def MLP_score_2(conf: Union[Type[ConfigurationSpace], dict]) -> float:
    """
    Basically takes a configuration does a 10-CV and return the average
    score, the average score is modified so it gets minimized, for example
    accuracy is converted into error.
    PARAMETERS
    ----------
    @conf   is a ConfigurationSpace Object, but it acts as a dictionary
            so we can call a value given the key: conf['key']
            
    Returns
    -------
    @return a float value which needs to be minimized in this case cv error
    
    This function takes in the configuration and computes the error score
    using CV with 10 folds
    """
    
    MLPclf = MLPClassifier(hidden_layer_sizes=(conf['h1'],conf['h2'],), 
                           momentum = conf['momentum'],
                           learning_rate_init = conf['learning_rate_init'],
                           random_state=BIG_SEED)
    
    skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
    X = df.drop('Class', axis=1).values
    y = df.Class
    score = cross_val_score(MLPclf, X, y, cv=skf, scoring=score_metric)
    #print(conf['learning_rate_init'], conf['momentum'], np.mean(score))
    return 1 - np.mean(score)  # it needs to Minimize it!

In [ ]:
#Use rf and EI
smac_rf_EI = optimizer('rf', cs, obj_function=MLP_score_2, 
                    acquisition_type='EI', n_iter=110, 
                    init_points=10, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_rf_EI.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Use rf and LCB
smac_rf_LCB = optimizer('rf', cs, obj_function=MLP_score_2, 
                    acquisition_type='LCB', n_iter=110, 
                    init_points=10, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_rf_LCB.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Use rf and PI
smac_rf_PI = optimizer('rf', cs, obj_function=MLP_score_2, 
                    acquisition_type='PI', n_iter=110, 
                    init_points=10, seed=BIG_SEED)
#plot the value for each iteration done
plt.plot(1 - smac_rf_PI.get_X_y()[1])
plt.ylabel(label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
#Compare the two models removing the initial 5 points keeping only the best of them.
plt.figure(figsize=(15,5))
plt.plot(1-np.minimum.accumulate(smac_rf_LCB.get_X_y()[1])[9:], 'o-') #0,1,2,3,4 sono i punti iniziali
plt.plot(1-np.minimum.accumulate(smac_rf_EI.get_X_y()[1])[9:], 'o-')
plt.plot(1-np.minimum.accumulate(smac_rf_PI.get_X_y()[1])[9:], 'o-')
plt.legend(['LCB','EI', 'PI'])
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.show()

## Random and Grid Search

In [ ]:
#create the configuration grid with 5 element for each hyperparameter
#We will obtain 125 iteration but during the plot we will plot only 
# the last 110 just so each model have the same lenght in terms of iterations.
conf_grid = {'learning_rate_init':np.linspace(0.01, 0.1, num=5),
             'momentum':np.linspace(0.1, 0.9, num=5),
             'hidden_layer_sizes':[(1,1), (1,5), (3,3), (5,1), (5,5)]}

MLP_GS = GridSearchCV(MLPClassifier(hidden_layer_sizes=(4,2,),random_state=BIG_SEED),
                      conf_grid, scoring=score_metric, cv=10, n_jobs=-1)

t_0 = time()
skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
X = df.drop('Class', axis=1).values
y = df.Class
MLP_GS = MLP_GS.fit(X, y)
print(MLP_GS.best_params_, "-> score", MLP_GS.best_score_)
print("Completed in %0.3fs" % (time() - t_0))

In [ ]:
hidden =[(i, j) for i in range(1,6) for j in range(1,6)]        
        
param_grid = {'learning_rate_init':np.linspace(0.01, 0.1),
              'momentum':np.linspace(0.1, 0.9),
              'hidden_layer_sizes':hidden}

MLP_RS = RandomizedSearchCV(MLPClassifier(hidden_layer_sizes=(4,2,), 
                                          random_state=BIG_SEED),
                            param_grid, n_iter=110, scoring=score_metric, 
                            cv=10, random_state=BIG_SEED, n_jobs=-1)

t_0 = time()
skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
X = df.drop('Class', axis=1).values
y = df.Class
MLP_RS = MLP_RS.fit(X, y)
print(MLP_RS.best_params_, "-> score", MLP_RS.best_score_)
print("Completed in %0.3fs" % (time() - t_0))

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(np.maximum.accumulate(MLP_RS.cv_results_['mean_test_score']), 'o-')
plt.plot(np.maximum.accumulate(MLP_GS.cv_results_['mean_test_score']), 'o-')
plt.legend(['Random','Grid'])
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
iter_num = list(range(1,111))
lcb, = plt.plot(iter_num, 1-np.minimum.accumulate(smac_rf_LCB.get_X_y()[1]), linewidth=4)
ei, = plt.plot(iter_num, 1-np.minimum.accumulate(smac_rf_EI.get_X_y()[1]),'*-', linewidth=3)
rs, = plt.plot(iter_num, np.maximum.accumulate(MLP_RS.cv_results_['mean_test_score']), linewidth=2)
gs, = plt.plot(iter_num, np.maximum.accumulate(MLP_GS.cv_results_['mean_test_score'])[15:], linewidth=2)
pi, = plt.plot(iter_num, 1-np.minimum.accumulate(smac_rf_PI.get_X_y()[1]), linewidth=2)

plt.legend([ei,rs,pi,lcb,gs], ['EI','Random','PI','LCB','Grid'])
plt.ylabel('Best Seen '+label_metric)
plt.xlabel('Iterations')
plt.xlim(xmin=0, xmax=111.5)
plt.show()

Also in this case the we see the best model found using LCB.

In [ ]:
best_2 = smac_rf_LCB.get_X_y()[0][np.argmin(smac_rf_LCB.get_X_y()[1])]

In [ ]:
MLPclf = MLPClassifier(hidden_layer_sizes=(int(best_2[0]), int(best_2[1])),
                      learning_rate_init=best_2[2], momentum=best_2[3],
                      random_state=BIG_SEED)

skf = StratifiedKFold(n_splits=10, random_state=BIG_SEED)
scores = cross_validate(MLPclf, df.drop('Class', axis=1).values, df.Class, cv=skf,
                        scoring=['f1','precision','recall','f1_macro','accuracy'])

In [ ]:
print("F1:\t\t %0.3f" % (np.mean(scores['test_f1'])))
print("Precision:\t %0.3f" % (np.mean(scores['test_precision'])))
print("Recall:\t\t %0.3f" % (np.mean(scores['test_recall'])))
print("F1 Macro:\t %0.3f" % (np.mean(scores['test_f1_macro'])))
print("Accuracy:\t %0.3f" % (np.mean(scores['test_accuracy'])))

In [ ]:
best_2

Even though the weighted accuracy imporves the overall performance on the rare class is decreased, so using RF as a surrogate models even with more iterations doesn't yield better results in this case.